https://habr.com/ru/articles/830396/

https://huggingface.co/facebook/opt-350m

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
from datasets import load_dataset

In [16]:
from trl import SFTTrainer, SFTConfig

In [23]:
import torch
import torch.backends.cuda

## load

In [4]:
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m", clean_up_tokenization_spaces=True)

In [5]:
model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")

## sample

In [6]:
x = "Hello world, ..."

In [7]:
inputs = tokenizer(x, return_tensors="pt")

In [8]:
inputs

{'input_ids': tensor([[    2, 31414,   232,     6,  1666]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [9]:
model_outputs = model.generate(**inputs, max_new_tokens=5, return_dict_in_generate=True, output_scores=True)

In [10]:
model_outputs

GenerateDecoderOnlyOutput(sequences=tensor([[    2, 31414,   232,     6,  1666, 50118, 50118,   100,   524,    10]]), scores=(tensor([[-4.7037, -4.6274,  7.3930,  ..., -5.8140, -4.0261, -5.8137]]), tensor([[ 6.8867,  6.3611, 16.4410,  ...,  5.7755,  6.8701,  5.1706]]), tensor([[ 8.3802,  8.1530, 12.3480,  ...,  8.8344,  9.1047,  8.5551]]), tensor([[-3.0737, -2.4380,  4.4715,  ..., -4.7422, -6.7250, -2.9836]]), tensor([[-19.6251, -18.9874, -12.6630,  ..., -19.9378, -20.2679, -17.3221]])), logits=None, attentions=None, hidden_states=None, past_key_values=((tensor([[[[ 0.3267, -0.3466, -0.4174,  ...,  0.3120, -0.1403, -0.3449],
          [-0.8922,  0.8802,  0.8702,  ..., -0.8283,  0.7369,  0.8701],
          [-0.8554,  0.8466,  0.8191,  ..., -0.8056,  0.7275,  0.8099],
          ...,
          [-0.6321,  0.6334,  0.6209,  ..., -0.6201,  0.5421,  0.6116],
          [-0.6520,  0.6009,  0.6235,  ..., -0.5771,  0.5319,  0.5837],
          [-0.6309,  0.6241,  0.6215,  ..., -0.5794,  0.5709,  0

In [11]:
generated_tokens_ids = model_outputs.sequences[0]

In [12]:
generated_tokens_ids

tensor([    2, 31414,   232,     6,  1666, 50118, 50118,   100,   524,    10])

In [13]:
tokenizer.decode(generated_tokens_ids)

'</s>Hello world,...\n\nI am a'

## train

In [14]:
dataset = load_dataset("imdb", split="train")

https://huggingface.co/docs/trl/en/sft_trainer

In [18]:
sft_config = SFTConfig(
    dataset_text_field="text",
    max_seq_length=512,
    output_dir="/tmp",
)

In [19]:
trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    args=sft_config,
)    

/home/nick/.conda/envs/transformers/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [24]:
with torch.backends.cuda.sdp_kernel(enable_flash=True, enable_math=False, enable_mem_efficient=False):
    trainer.train()

/home/nick/.conda/envs/transformers/lib/python3.10/contextlib.py:103: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)


OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacity of 11.73 GiB of which 9.06 MiB is free. Including non-PyTorch memory, this process has 11.61 GiB memory in use. Of the allocated memory 11.43 GiB is allocated by PyTorch, and 14.82 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)